In [1]:
import re
import numpy as np
import plotly.graph_objects as go

In [235]:
import re

def parse_heart_exf(file_path):
    """
    Parses a heart.exf file with multiple regions.
    
    For each region (indicated by "Region:"), this function extracts:
      - nodes: by looking for "Node:" lines and reading the next three lines,
               but only if those lines start with a number.
      - mesh3d connectivity: by detecting the "!#mesh mesh3d" block and then extracting 
        each element's node IDs (after a "Nodes:" label).
        
    If a "Node:" line is found but its next three lines do not contain valid numbers,
    the node is recorded as "empty" and only the node header is skipped.
    
    Returns:
        A dictionary with region names as keys. Each value is another dictionary with keys:
          "nodes": a list of node dictionaries, e.g.
              [{'node': 1, 'x': ..., 'y': ..., 'z': ...}, ...]
          "mesh3d": a list of element dictionaries, e.g.
              [{'element': 7, 'nodes': [1,8,9,108,115,116]}, ...]
          "empty_nodes": a list of node numbers that did not have valid coordinates.
    """
    
    def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            return False

    regions = {}
    current_region = None  # current region name
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        
        # New region header
        if line.startswith("Region:"):
            current_region = line.split(":", 1)[1].strip()
            if current_region not in regions:
                regions[current_region] = {"nodes": [], "mesh3d": [], "empty_nodes": []}
            i += 1
            continue
        
        # Parse node blocks
        if line.startswith("Node:"):
            match = re.match(r"Node:\s*(\d+)", line)
            node_number = int(match.group(1)) if match else None
            coords = []
            valid = True
            # Check the next 3 lines for valid numbers
            for j in range(1, 4):
                if i + j < len(lines):
                    coord_line = lines[i+j].strip()
                    parts = coord_line.split()
                    if parts and is_number(parts[0]):
                        try:
                            coords.append(float(parts[0]))
                        except ValueError:
                            valid = False
                            break
                    else:
                        valid = False
                        break
            if valid and len(coords) == 3:
                # Ensure a region exists
                if current_region is None:
                    current_region = "default"
                    regions[current_region] = {"nodes": [], "mesh3d": [], "empty_nodes": []}
                regions[current_region]["nodes"].append({
                    'node': node_number,
                    'x': coords[0],
                    'y': coords[1],
                    'z': coords[2]
                })
                i += 4  # Skip node header and next 3 coordinate lines
            else:
                # Not valid – record the node as empty and only skip the node header line.
                if current_region is None:
                    current_region = "default"
                    regions[current_region] = {"nodes": [], "mesh3d": [], "empty_nodes": []}
                regions[current_region]["empty_nodes"].append(node_number)
                i += 1
            continue
        
        # Parse mesh3d connectivity
        if line.startswith("!#mesh mesh3d"):
            if current_region is None:
                current_region = "default"
                regions[current_region] = {"nodes": [], "mesh3d": [], "empty_nodes": []}
            i += 1  # Move past the header
            while i < len(lines) and not lines[i].strip().startswith("Region:"):
                subline = lines[i].strip()
                if subline.startswith("Element:"):
                    match = re.match(r"Element:\s*(\d+)", subline)
                    if match:
                        elem_id = int(match.group(1))
                        connectivity = []
                        found_nodes = False
                        i += 1
                        while i < len(lines):
                            subline = lines[i].strip()
                            if subline.startswith("Nodes:"):
                                found_nodes = True
                                i += 1  # Move to the line with node IDs
                                connectivity = [int(x) for x in lines[i].strip().split()]
                                break
                            else:
                                i += 1
                        if found_nodes and len(connectivity) in [4, 5, 6, 7, 8]:
                            regions[current_region]["mesh3d"].append({
                                'element': elem_id,
                                'nodes': connectivity
                            })
                        continue
                else:
                    i += 1
            continue
        
        # Default: move to next line.
        i += 1
    
    return regions


In [237]:
file_path = "data/heart.exf"  # Replace with the correct path to your file
heart_data = parse_heart_exf(file_path)

# Print summary information
for region, data in heart_data.items():
    print(f"Region: {region}")
    print("  Number of nodes:", len(data["nodes"]))
    print("  Number of mesh3d elements:", len(data["mesh3d"]))
    print("  Number of empty nodes:", len(data["empty_nodes"]))
    print()


Region: /
  Number of nodes: 639
  Number of mesh3d elements: 345
  Number of empty nodes: 7

Region: /systemic arteries
  Number of nodes: 1638
  Number of mesh3d elements: 0
  Number of empty nodes: 0

Region: /systemic veins
  Number of nodes: 992
  Number of mesh3d elements: 0
  Number of empty nodes: 0



In [239]:
[data["empty_nodes"] for region, data in heart_data.items()]

[[165, 224, 225, 497, 558, 617, 646], [], []]

In [240]:
heart_nodes = heart_data['/']['nodes']

In [241]:
heart_nodes[-1]

{'node': 645,
 'x': -12.41524165421417,
 'y': -139.543265148779,
 'z': 1262.138453500766}

In [242]:
heart_mesh = heart_data['/']['mesh3d']

In [243]:
# Extract coordinate lists for plotting
x_vals = [node['x'] for node in heart_nodes]
y_vals = [node['y'] for node in heart_nodes]
z_vals = [node['z'] for node in heart_nodes]

# Compute a unified axis range using the min and max of all coordinate values
global_min = min(min(x_vals), min(y_vals), min(z_vals))
global_max = max(max(x_vals), max(y_vals), max(z_vals))
abs_max = max(abs(global_min), abs(global_max))
axis_range = [-abs_max, abs_max]

# Create a 3D scatter plot using Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=x_vals,
    y=y_vals,
    z=z_vals,
    mode='markers',
    marker=dict(size=2)
)])
fig.update_layout(
    title='3D Scatter Plot of SPARC Human Wholebody Scaffold',
    scene=dict(
        xaxis=dict(title='X', 
                   #range=axis_range
                   ),
        yaxis=dict(title='Y',
                   #range=axis_range
                   ),
        zaxis=dict(title='Z',
                   #range=[range + 800 for range in axis_range]
                   ),
        aspectmode='cube'  # Ensures equal scale for x, y, z axes
    ),
    width=800,   # Increase width as needed
    height=800   # Increase height as needed
)

fig.show()

In [244]:
[elem for elem in heart_mesh if len(elem['nodes'])==7]

[{'element': 92, 'nodes': [44, 45, 193, 151, 152, 190, 211]},
 {'element': 107, 'nodes': [101, 182, 183, 151, 190, 188, 189]},
 {'element': 117, 'nodes': [57, 44, 169, 170, 175, 190, 176]},
 {'element': 119, 'nodes': [193, 192, 170, 171, 211, 177, 176]},
 {'element': 126, 'nodes': [206, 103, 207, 104, 221, 174, 173]}]

In [245]:
# Define the special node IDs
special_ids = {206, 103, 207, 104, 221, 174, 173}

# make experimental mesh
test_triangles = [[173,206,207],
                            [173,206,221],
                            [173,174,221],
                            [103,174,221],
                            [103,173,174],
                            [103,104,221],
                            [104,206,221],
                            [104,206,207],
                            [103,104,207],
                            [103,173,207]]
i_mesh = []
j_mesh = []
k_mesh = []
for tri in test_triangles:
    # Subtract 1 from each node ID to convert to zero-indexing.
    i_mesh.append(sorted(special_ids).index(tri[0]))
    j_mesh.append(sorted(special_ids).index(tri[1]))
    k_mesh.append(sorted(special_ids).index(tri[2]))
test_nodes = [elem for elem in heart_nodes if elem['node'] in special_ids]
test_nodes_dict = {node_entry['node']: (node_entry['x'], node_entry['y'], node_entry['z']) for node_entry in test_nodes}
test_num_nodes = max(test_nodes_dict.keys())
x_mesh = [test_nodes_dict[i][0] for i in test_nodes_dict.keys()]
y_mesh = [test_nodes_dict[i][1] for i in test_nodes_dict.keys()]
z_mesh = [test_nodes_dict[i][2] for i in test_nodes_dict.keys()]
test_mesh = go.Mesh3d(
    x=x_mesh,
    y=y_mesh,
    z=z_mesh,
    i=i_mesh,
    j=j_mesh,
    k=k_mesh,
    opacity=0.5,
    color='lightblue'
)


# Separate nodes into normal and special based on their 'node' field
normal_nodes = [node for node in heart_nodes if node['node'] not in special_ids]
special_nodes = [node for node in heart_nodes if node['node'] in special_ids]

# Extract coordinate lists for normal nodes
x_normal = [node['x'] for node in normal_nodes]
y_normal = [node['y'] for node in normal_nodes]
z_normal = [node['z'] for node in normal_nodes]
text_normal = [f"Node: {node['node']}" for node in normal_nodes]

# Extract coordinate lists for special nodes
x_special = [node['x'] for node in special_nodes]
y_special = [node['y'] for node in special_nodes]
z_special = [node['z'] for node in special_nodes]
text_special = [f"Node: {node['node']}" for node in special_nodes]

# Compute a unified axis range using the min and max of all coordinate values
global_min = min(min([node['x'] for node in heart_nodes]),
                 min([node['y'] for node in heart_nodes]),
                 min([node['z'] for node in heart_nodes]))
global_max = max(max([node['x'] for node in heart_nodes]),
                 max([node['y'] for node in heart_nodes]),
                 max([node['z'] for node in heart_nodes]))
abs_max = max(abs(global_min), abs(global_max))
axis_range = [-abs_max, abs_max]

# Create the 3D scatter plot with two traces
fig = go.Figure()

# Normal nodes
fig.add_trace(go.Scatter3d(
    x=x_normal,
    y=y_normal,
    z=z_normal,
    mode='markers',
    marker=dict(size=2, color='blue'),
    text=text_normal,
    hoverinfo='text',
    name='Normal Nodes'
))

# Special nodes (different colour and slightly larger)
fig.add_trace(go.Scatter3d(
    x=x_special,
    y=y_special,
    z=z_special,
    mode='markers',
    marker=dict(size=4, color='red'),
    text=text_special,
    hoverinfo='text',
    name='Special Nodes'
))

fig.add_trace(test_mesh)

# Update the layout
fig.update_layout(
    title='3D Scatter Plot with Special Nodes Highlighted',
    scene=dict(
        xaxis=dict(title='X', 
                   #range=axis_range
                   ),
        yaxis=dict(title='Y',
                   #range=axis_range
                   ),
        zaxis=dict(title='Z',
                   #range=[range + 800 for range in axis_range]
                   ),
        aspectmode='cube'  # Ensures equal scale for x, y, z axes
    ),
    width=800,
    height=800
)

fig.show()

In [246]:
special_triangles = {92: [[44,45,152],#4-edge-base
                           [44,151,152],#4-edge-base
                           [44,45,193],
                           [151,152,190],
                           [44,151,190],
                           [45,152,193],
                           [152,193,211],
                           [152,190,211],
                           [44,193,211]],
                     107: [[101,183,189],#4-edge-base
                            [101,189,190],#4-edge-base
                            [101,151,190],
                            [151,188,190],
                            [182,183,188],
                            [183,188,189],
                            [188,189,190],
                            [101,182,183],
                            [101,182,188],
                            [101,151,188]],
                     117: [[57,169,170],
                            [57,169,175],
                            [57,170,176],
                            [44,57,175],
                            [44,57,170],
                            [44,170,176],
                            [44,176,190],
                            [44,175,190],
                            [175,176,190],
                            [169,170,175],
                            [170,175,176]],
                     119: [[176,193,211],
                            [176,192,193],
                            [177,193,211],
                            [177,192,193],
                            [176,177,211],
                            [170,176,177],
                            [170,171,177],
                            [171,177,192],
                            [170,171,192],
                            [170,176,192]],
                     126: [[173,206,207],
                            [173,206,221],
                            [173,174,221],
                            [103,174,221],
                            [103,173,174],
                            [103,104,221],
                            [104,206,221],
                            [104,206,207],
                            [103,104,207],
                            [103,173,207]]}

# Heart mesh

In [247]:
def element_to_triangles(element_nodes, elem_id, exceptions_for_7):
    """
    Converts a mesh3d element into triangles based on the number of nodes.
    
    - 4 nodes: Assumes a tetrahedron.
    - 5 nodes: Assumes a pyramid with a quadrilateral base.
    - 6 nodes: Assumes a wedge element.
    - 8 nodes: Assumes a hexahedron.
    - 7 nodes: For a 7-node element, if an exception is provided,
    the corresponding manual triangulation is returned (relative indices).
    
    
    Returns:
         A list of triangles (each triangle is a list of 3 node indices).
    """
    n = len(element_nodes)
    if n == 4:
        # Tetrahedron: 4 triangular faces.
        return [
            [element_nodes[0], element_nodes[1], element_nodes[2]],
            [element_nodes[0], element_nodes[1], element_nodes[3]],
            [element_nodes[0], element_nodes[2], element_nodes[3]],
            [element_nodes[1], element_nodes[2], element_nodes[3]],
        ]
    elif n == 5:
        # Pyramid: Assume ordering [base0, base1, base2, base3, apex]
        triangles = []
        # Base: quadrilateral split into two triangles.
        triangles.append([element_nodes[0], element_nodes[1], element_nodes[2]])
        triangles.append([element_nodes[0], element_nodes[2], element_nodes[3]])
        # Four triangular side faces.
        triangles.append([element_nodes[0], element_nodes[1], element_nodes[4]])
        triangles.append([element_nodes[1], element_nodes[2], element_nodes[4]])
        triangles.append([element_nodes[2], element_nodes[3], element_nodes[4]])
        triangles.append([element_nodes[3], element_nodes[0], element_nodes[4]])
        return triangles
    elif n == 6:
        # Wedge element: Assume ordering [n0, n1, n2, n3, n4, n5] where
        # Bottom triangle: [n0, n1, n2], Top triangle: [n3, n4, n5]
        triangles = []
        # Bottom and top faces.
        triangles.append([element_nodes[0], element_nodes[1], element_nodes[2]])
        triangles.append([element_nodes[3], element_nodes[4], element_nodes[5]])
        # Side face 1: quad [n0, n1, n4, n3]
        triangles.append([element_nodes[0], element_nodes[1], element_nodes[4]])
        triangles.append([element_nodes[0], element_nodes[4], element_nodes[3]])
        # Side face 2: quad [n1, n2, n5, n4]
        triangles.append([element_nodes[1], element_nodes[2], element_nodes[5]])
        triangles.append([element_nodes[1], element_nodes[5], element_nodes[4]])
        # Side face 3: quad [n2, n0, n3, n5]
        triangles.append([element_nodes[2], element_nodes[0], element_nodes[3]])
        triangles.append([element_nodes[2], element_nodes[3], element_nodes[5]])
        return triangles
    elif n == 7:
        if exceptions_for_7 is None:
            raise ValueError(f"7-node element triangulation is not defined.")
        # Return the manually specified triangulation.
        # Optionally, if you need absolute node IDs, you can map the relative indices.
        return exceptions_for_7[elem_id]
    elif n == 8:
        # Hexahedron: Assume ordering with bottom face first, then top face.
        faces = [
            [element_nodes[0], element_nodes[1], element_nodes[2], element_nodes[3]],  # bottom face
            [element_nodes[4], element_nodes[5], element_nodes[6], element_nodes[7]],  # top face
            [element_nodes[0], element_nodes[1], element_nodes[5], element_nodes[4]],  # side face 1
            [element_nodes[1], element_nodes[2], element_nodes[6], element_nodes[5]],  # side face 2
            [element_nodes[2], element_nodes[3], element_nodes[7], element_nodes[6]],  # side face 3
            [element_nodes[3], element_nodes[0], element_nodes[4], element_nodes[7]],  # side face 4
        ]
        triangles = []
        for quad in faces:
            triangles.append([quad[0], quad[1], quad[2]])
            triangles.append([quad[0], quad[2], quad[3]])
        return triangles
    elif n == 7:
        raise ValueError("7-node element triangulation is not implemented")
    else:
        raise ValueError("Unsupported element type: expected 4,5,6,7 or 8 nodes, got {}".format(n))

In [248]:
nodes_dict = {node_entry['node']: (node_entry['x'], node_entry['y'], node_entry['z']) for node_entry in heart_nodes}

print(nodes_dict)

{1: (62.35580111952793, -157.7114651727438, 1183.291909945507), 2: (59.63074934588204, -161.9963186134961, 1193.749636387601), 3: (64.37549377209234, -158.7368855887872, 1194.695241715899), 4: (68.18392512464033, -152.6280263241288, 1191.972746717762), 5: (67.31725456047714, -147.6084720936347, 1186.203005334865), 6: (62.24854909080931, -146.4176097320749, 1180.534891225542), 7: (55.74398166379608, -149.7053351673337, 1178.061695967987), 8: (51.35343821770641, -155.6774305707127, 1180.133240196425), 9: (50.97743446749064, -159.9632733710865, 1184.073008621493), 10: (51.78174337917491, -161.4875886827205, 1186.303522895189), 11: (52.55357898828932, -162.1722997855922, 1187.668401331232), 12: (53.51593121356927, -162.6534335305076, 1188.998779883713), 13: (54.64114467773575, -162.9177957931979, 1190.25705743411), 14: (55.90205984030951, -162.9590055991615, 1191.412862642964), 15: (57.2631817130937, -162.7754202658678, 1192.433178306648), 16: (53.67889890611039, -157.4973965204463, 1208.9

In [249]:
# Convert all elements into triangles.
all_triangles = []
for elem in heart_mesh:
    elem_nodes = elem['nodes']
    elem_id = elem['element']
    # Convert the hexahedron or wedges or others into a list of triangles.
    triangles = element_to_triangles(elem_nodes, elem_id, special_triangles)
    all_triangles.extend(triangles)

In [250]:
len(all_triangles)

3880

In [251]:
all_nodes = [elem['node'] for elem in heart_nodes]

In [253]:
all_nodes.index(226)

222

In [254]:
# --- Step 3: Prepare the Triangle Indices for Plotly's Mesh3d ---
# Plotly requires the triangle vertex indices (i, j, k) to be zero-indexed.
i_vals = []
j_vals = []
k_vals = []
for tri in all_triangles:
    # Subtract 1 from each node ID to convert to zero-indexing.
    i_vals.append(all_nodes.index(tri[0]))
    j_vals.append(all_nodes.index(tri[1]))
    k_vals.append(all_nodes.index(tri[2]))

# --- Step 4: Prepare Node Coordinates ---
# Build lists for the x, y, z coordinates. We assume the node IDs in nodes_dict are consecutive.
#num_nodes = max(nodes_dict.keys())
x_all = [nodes_dict[i][0] for i in nodes_dict.keys()]
y_all = [nodes_dict[i][1] for i in nodes_dict.keys()]
z_all = [nodes_dict[i][2] for i in nodes_dict.keys()]



In [255]:
min(i_vals)

0

In [256]:
max(i_vals)

638

In [257]:
len(x_all)

639

In [265]:
# Create a Mesh3d plot with the parsed coordinates and connectivity.
mesh = go.Mesh3d(
    x=x_all,
    y=y_all,
    z=z_all,
    i=i_vals,
    j=j_vals,
    k=k_vals,
    opacity=0.5,
    color='lightblue'
)

fig = go.Figure(data=[mesh])

fig.add_trace(go.Scatter3d(
    x=x_all,
    y=y_all,
    z=z_all,
    mode='markers',
    marker=dict(size=2, color='blue'),
    text=text_normal,
    hoverinfo='text',
    name='Normal Nodes'
))

fig.update_layout(
    title="3D Mesh using Connectivity from exf File",
    scene=dict(
        xaxis=dict(title='X', 
                   #range=axis_range
                   ),
        yaxis=dict(title='Y',
                   #range=axis_range
                   ),
        zaxis=dict(title='Z',
                   #range=[range + 800 for range in axis_range]
                   ),
        aspectmode="cube"
    ),
    width=800,
    height=800
)
fig.show()

# Save as CSV

In [267]:
import pandas as pd

In [268]:
node_df = pd.DataFrame(heart_nodes).set_index("node")
node_df

,x,y,z
node,,,
1,62.355801,-157.711465,1183.291910
2,59.630749,-161.996319,1193.749636
3,64.375494,-158.736886,1194.695242
4,68.183925,-152.628026,1191.972747
5,67.317255,-147.608472,1186.203005
...,...,...,...
641,-8.949739,-122.400048,1257.442244
642,-14.789559,-123.144317,1257.433698
643,-15.704112,-129.151342,1260.588804


In [269]:
mesh_df = pd.DataFrame({"triangle": list(range(len(i_vals))),
                               "i": i_vals,
                               "j": j_vals,
                               "k": k_vals}).set_index("triangle")
mesh_df

,i,j,k
triangle,,,
0,0,1,2
1,107,108,109
2,0,1,108
3,0,108,107
4,1,2,109
...,...,...,...
3875,189,348,206
3876,340,330,347
3877,340,347,348


In [270]:
node_df.to_csv("data/heart_nodes_all.csv", index=True)
mesh_df.to_csv("data/heart_mesh_triangles.csv", index=True)